In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import seaborn as sns

sns.set_theme("notebook",style="dark")

In [ ]:
import datetime
from pathlib import Path

import torch

from qgsw.cli import ScriptArgs
from qgsw.configs.core import Configuration
from qgsw.fields.variables.tuples import UVH
from qgsw.forcing.wind import WindForcing
from qgsw.logging import getLogger, setup_root_logger
from qgsw.masks import Masks
from qgsw.models.qg.psiq.core import QGPSIQ
from qgsw.models.qg.psiq.filtered.core import (
    QGPSIQCollinearSF,
    QGPSIQFixeddSF2,
)
from qgsw.models.qg.psiq.optim.utils import EarlyStop, RegisterParams
from qgsw.models.qg.stretching_matrix import compute_A
from qgsw.models.qg.uvh.projectors.core import QGProjector
from qgsw.solver.boundary_conditions.base import Boundaries
from qgsw.spatial.core.discretization import (
    SpaceDiscretization2D,
    SpaceDiscretization3D,
)
import matplotlib.pyplot as plt
from qgsw.specs import defaults
from qgsw.utils import covphys
from qgsw.utils.interpolation import QuadraticInterpolation

torch.backends.cudnn.deterministic = True
torch.set_grad_enabled(False)
import gc

specs = defaults.get()

setup_root_logger(1)
logger = getLogger(__name__)

config = Configuration.from_toml("../output/g5k/param_optim/_config.toml")

output_dir = config.io.output.directory

# Parameters

H = config.model.h
g_prime = config.model.g_prime
beta_plane = config.physics.beta_plane
bottom_drag_coef = config.physics.bottom_drag_coefficient
slip_coef = config.physics.slip_coef

space = SpaceDiscretization3D.from_config(
    config.space,
    config.model,
)
P = QGProjector(
    A=compute_A(H=H, g_prime=g_prime),
    H=H.unsqueeze(-1).unsqueeze(-1),
    space=space,
    f0=beta_plane.f0,
    masks=Masks.empty(
        nx=config.space.nx,
        ny=config.space.ny,
    ),
)
dx, dy = space.dx, space.dy
nx, ny = space.nx, space.ny

wind = WindForcing.from_config(
    config.windstress,
    config.space,
    config.physics,
)
tx, ty = wind.compute()

uvh0 = UVH.from_file("../output/g5k/param_optim/_data_startup.pt")
psi_start = P.compute_p(covphys.to_cov(uvh0, dx, dy))[0] / beta_plane.f0

## Areas

imins = [32, 32, 112, 112]
i_len = 64
imaxs = [i + i_len for i in imins]

jmins = [64, 256, 64, 256]
j_len = 128
jmaxs = [j + j_len for j in jmins]


def compute_slices(
    imin: int, imax: int, jmin: int, jmax: int
) -> tuple[list[slice, slice], list[slice, slice]]:
    """Compute horizontal slices."""
    psi_slices = [slice(imin, imax + 1), slice(jmin, jmax + 1)]
    q_slices = [slice(imin, imax), slice(jmin, jmax)]

    return psi_slices, q_slices


n_areas = len(imins)
str_area_len = len(str(n_areas))

## Simulation parameters

dt = 3600
optim_max_step = 100
str_optim_len = len(str(optim_max_step))
n_steps_per_cyle = 500
comparison_interval = 1

## Error


def rmse(f: torch.Tensor, f_ref: torch.Tensor) -> float:
    """RMSE."""
    return (f - f_ref).square().mean().sqrt() / f_ref.square().mean().sqrt()


# Models
## Three Layer model

model_3l = QGPSIQ(
    space_2d=space.remove_z_h(),
    H=H,
    beta_plane=config.physics.beta_plane,
    g_prime=g_prime,
)
model_3l.set_wind_forcing(tx, ty)
model_3l.masks = Masks.empty_tensor(
    model_3l.space.nx,
    model_3l.space.ny,
    device=specs["device"],
)
model_3l.bottom_drag_coef = bottom_drag_coef
model_3l.slip_coef = slip_coef
model_3l.dt = dt
y0 = model_3l.y0


## Inhomogeneous models
def set_inhomogeneous_model(
    model: QGPSIQ | QGPSIQCollinearSF | QGPSIQFixeddSF2,
) -> QGPSIQ | QGPSIQCollinearSF | QGPSIQFixeddSF2:
    """Set up inhomogeneous model."""
    space = model.space
    model.y0 = y0
    model.masks = Masks.empty_tensor(
        space.nx,
        space.ny,
        device=specs["device"],
    )
    model.bottom_drag_coef = 0
    model.wide = True
    model.slip_coef = slip_coef
    model.dt = dt
    return model

In [ ]:
output_files = [
    "../output/g5k/param_optim/results_32_96_64_192.pt",
    "../output/g5k/param_optim/results_32_96_256_384.pt",
    "../output/g5k/param_optim/results_112_176_64_192.pt",
    "../output/g5k/param_optim/results_112_176_256_384.pt"
]

outputs = {
    "indices": [],
    "losses_1l": [],
    "losses_2l": [],
    "losses_alpha": [],
    "losses_dpsi": [],
    "alphas": [],
    "dpsis": []
}

for f in output_files:

    optim_results = torch.load(f)
    indices = optim_results[0]["coords"]
    n_cycles = len(optim_results)

    logger.info(f"Indices: {indices}")
    
    outputs["indices"].append(indices)

    imin, imax, jmin, jmax = indices

    model_3l.reset_time()
    model_3l.set_psi(psi_start)

    psi_slices, q_slices = compute_slices(imin, imax, jmin, jmax)

    space_slice = SpaceDiscretization2D.from_tensors(
        x=P.space.remove_z_h().omega.xy.x[imin : imax + 1, 0],
        y=P.space.remove_z_h().omega.xy.y[0, jmin : jmax + 1],
    )

    model_1l = QGPSIQ(
        space_2d=space_slice,
        H=H[:1]*H[1:2]/(H[:1]+H[1:2]),
        beta_plane=beta_plane,
        g_prime=g_prime[1:2],
    )
    model_2l = QGPSIQ(
        space_2d=space_slice,
        H=H[:2],
        beta_plane=beta_plane,
        g_prime=g_prime[:2],
    )
    model_alpha = QGPSIQCollinearSF(
        space_2d=space_slice,
        H=H[:2],
        beta_plane=beta_plane,
        g_prime=g_prime[:2],
    )
    model_dpsi = QGPSIQFixeddSF2(
        space_2d=space_slice,
        H=H[:2],
        beta_plane=beta_plane,
        g_prime=g_prime[:2],
    )
    model_1l: QGPSIQ = set_inhomogeneous_model(model_1l)
    model_2l: QGPSIQ = set_inhomogeneous_model(model_2l)
    model_alpha: QGPSIQCollinearSF = set_inhomogeneous_model(model_alpha)
    model_dpsi: QGPSIQFixeddSF2 = set_inhomogeneous_model(model_dpsi)

    model_alpha.set_wind_forcing(
        tx[imin:imax, jmin : jmax + 1], ty[imin : imax + 1, jmin:jmax]
    )
    model_dpsi.set_wind_forcing(
        tx[imin:imax, jmin : jmax + 1], ty[imin : imax + 1, jmin:jmax]
    )

    def extract_psi(psi: torch.Tensor) -> tuple[torch.Tensor, Boundaries]:
        """Extract psi."""
        return psi[..., psi_slices[0], psi_slices[1]], Boundaries.extract(
            psi, imin, imax + 1, jmin, jmax + 1, 2
        )

    def extract_q(q: torch.Tensor) -> tuple[torch.Tensor, Boundaries]:
        """Extract q."""
        return q[..., q_slices[0], q_slices[1]], Boundaries.extract(
            q, imin - 1, imax + 1, jmin - 1, jmax + 1, 3
        )

    losses_alpha = []
    losses_dpsi = []
    losses_2l = []
    losses_1l = []
    alphas = []
    dpsis = []

    for c in range(n_cycles):
        times = [model_3l.time.item()]

        psi0, psi_bc = extract_psi(model_3l.psi[:, :2])
        q0, q_bc = extract_q(model_3l.q[:, :2])

        psis = [psi0]
        psi_bcs = [psi_bc]
        qs = [q0]
        q_bcs = [q_bc]

        for _ in range(1, n_steps_per_cyle):
            model_3l.step()

            times.append(model_3l.time.item())

            psi, psi_bc = extract_psi(model_3l.psi[:, :2])
            q, q_bc = extract_q(model_3l.q[:, :2])

            psis.append(psi)
            psi_bcs.append(psi_bc)
            qs.append(q)
            q_bcs.append(q_bc)
            

        psi_bc_interp_1l = QuadraticInterpolation(times, [p[:,:1] for p in psi_bcs])
        q_bc_interp_1l = QuadraticInterpolation(times, [q[:,:1] for q in q_bcs])
        psi_bc_interp_2l = QuadraticInterpolation(times, psi_bcs)
        q_bc_interp_2l = QuadraticInterpolation(times, q_bcs)

        model_1l.reset_time()
        model_1l.set_psiq(psi0[:,:1], q0[:,:1])
        losses_1l.append(rmse(model_1l.psi[0, 0], psis[0][0, 0]).cpu().item())
        model_1l.set_boundary_maps(psi_bc_interp_1l, q_bc_interp_1l)

        for n in range(1, n_steps_per_cyle):
            model_1l.step()

            if (n + 1) % comparison_interval == 0:
                losses_1l.append(rmse(model_1l.psi[0, 0], psis[n][0, 0]).cpu().item())

        model_2l.reset_time()
        model_2l.set_psiq(psi0[:,:2], q0[:,:2])
        losses_2l.append(rmse(model_2l.psi[0, 0], psis[0][0, 0]).cpu().item())
        model_2l.set_boundary_maps(psi_bc_interp_2l, q_bc_interp_2l)

        for n in range(1, n_steps_per_cyle):
            model_2l.step()

            if (n + 1) % comparison_interval == 0:
                losses_2l.append(rmse(model_2l.psi[0, 0], psis[n][0, 0]).cpu().item())
        
        alpha:torch.Tensor = optim_results[c]["alpha"]
        alphas.append(alpha)
        model_alpha.reset_time()
        model_alpha.set_psiq(psi0[:,:1], q0[:,:1])
        losses_alpha.append(rmse(model_alpha.psi[0, 0], psis[0][0, 0]).cpu().item())
        model_alpha.alpha = torch.ones_like(psi0[:,:1]) * alpha
        model_alpha.set_boundary_maps(psi_bc_interp_1l, q_bc_interp_1l)

        for n in range(1, n_steps_per_cyle):
            model_alpha.step()

            if (n + 1) % comparison_interval == 0:
                losses_alpha.append(rmse(model_alpha.psi[0, 0], psis[n][0, 0]).cpu().item())

        dpsi2:torch.Tensor = optim_results[c]["dpsi2"]
        dpsis.append(dpsi2)
        model_dpsi.reset_time()
        model_dpsi.set_psiq(psi0[:,:1], q0[:,:1])
        losses_dpsi.append(rmse(model_dpsi.psi[0, 0], psis[0][0, 0]).cpu().item())
        model_dpsi.dpsi2 = dpsi2.to(**specs)
        model_dpsi.set_boundary_maps(psi_bc_interp_1l, q_bc_interp_1l)

        for n in range(1, n_steps_per_cyle):
            model_dpsi.step()

            if (n + 1) % comparison_interval == 0:
                losses_dpsi.append(rmse(model_dpsi.psi[0, 0], psis[n][0, 0]).cpu().item())
    outputs["alphas"].append(alphas)
    outputs["dpsis"].append(dpsis)
    outputs["losses_1l"].append(losses_1l)
    outputs["losses_2l"].append(losses_2l)
    outputs["losses_alpha"].append(losses_alpha)
    outputs["losses_dpsi"].append(losses_dpsi)

    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
from qgsw import plots

fig,axs = plots.subplots(len(output_files),2, gridspec_kw = {"width_ratios":[1,5]},figsize=(12,16))
for i,indices in enumerate(outputs["indices"]):
    imin,imax,jmin,jmax = indices

    plots.imshow(psi_start[0,0],ax=axs[i,0])
    axs[i,0].hlines([jmin,jmax],imin,imax,color="black")
    axs[i,0].vlines([imin,imax],jmin,jmax,color="black")


    axs[i,1].hlines(1,0,1500,linestyle="dashed",color="brown",alpha=0.75)
    axs[i,1].plot(outputs["losses_1l"][i],color="blue", label = "1 layer")
    axs[i,1].plot(outputs["losses_2l"][i],color="black", label = "2 layers")
    axs[i,1].plot(outputs["losses_alpha"][i],color="red", label = "ɑ")
    axs[i,1].plot(outputs["losses_dpsi"][i],color="orange", label = "dѱ2")
    axs[i,1].legend(loc="upper left")

plots.show()

In [ ]:
from qgsw import plots

fig,axs = plots.subplots(len(output_files), len(outputs["dpsis"][0])+1,figsize=(15,15))

for i,indices in enumerate(outputs["indices"]):
    imin,imax,jmin,jmax = indices

    plots.imshow(psi_start[0,0],ax=axs[i,0])

    for j, dpsi in enumerate(outputs["dpsis"][i]):
        plots.imshow(dpsi[0,0,1:-1,1:-1],ax=axs[i,j+1])

plots.show()